In [ ]:
from google.colab import files

uploaded = files.upload()

Saving spam.csv to spam.csv


Upload widget is only available when the cell has been executed in the current browser session. Please rerun this cell to enable.
Saving SMSSpamCollection to SMSSpamCollection


In [ ]:
import pandas as pd
import io

IMP - check file name and extension while using read_csv if you are using this method for uploading in this method

In [32]:
messages = pd.read_csv(io.BytesIO(uploaded['spam.csv']),sep='\t',names=["label", "message"], encoding = "ISO-8859-1")
messages2 = pd.read_csv(io.BytesIO(uploaded['spam.csv']), encoding = "ISO-8859-1")
# messages = pd.read_csv(io.BytesIO(uploaded['spam.csv']))

In [66]:
messages[:3]

,label,message
0,"v1,v2,,,",NaN
1,"ham,""Go until jurong point, crazy.. Available ...",NaN
2,"ham,Ok lar... Joking wif u oni...,,,",NaN


In [67]:
len(messages2)

5572

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [68]:
corpus = []

In [69]:
corpus

[]

In [70]:
messages2.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [71]:
for i in range(0, len(messages2)):
  review = re.sub('[^a-zA-Z]',' ', messages2['v2'][i])
  review = review.lower()
  review = review.split()

  # applying stemming
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [74]:
  corpus[:4]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say']

In [75]:
len(corpus)

5572

Creating a bag of words of the model

In [76]:
from sklearn.feature_extraction.text import CountVectorizer

In [77]:
cv = CountVectorizer(max_features = 5000) # take only 500 maximum features
# column heading is a word 
# every row is a sentence
# entry in the matrix = 1 if that word is present in the sentence
# otherwise 0

In [78]:
X = cv.fit_transform(corpus).toarray()

In [79]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [80]:
X.size

27860000

In [81]:
X.shape

(5572, 5000)

In [83]:
5572 * 5000

27860000

In [51]:
# Converting labels to numerical values

In [53]:
y = pd.get_dummies(messages2['v1'])

In [54]:
y[:4]

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0


In [55]:
y = y.iloc[:,1].values  # remove spam column

In [56]:
y.shape

(5572,)

In [84]:
y # ham -> 0 spam -> 1

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

Train Test Split

In [58]:
from sklearn.model_selection import train_test_split

In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3 , random_state = 0)

In [87]:
# Training model using Naive Bayes Classifier

In [88]:
X_train.shape

(3900, 5000)

In [89]:
X_test.shape

(1672, 5000)

In [90]:
from sklearn.naive_bayes import MultinomialNB

In [91]:
spam_detection_model = MultinomialNB().fit(X_train, Y_train)

In [93]:
y_pred = spam_detection_model.predict(X_test)

In [94]:
from sklearn.metrics import confusion_matrix

In [96]:
confusion_m = confusion_matrix(Y_test, y_pred)

In [97]:
confusion_m

array([[1413,   21],
       [   8,  230]])

In [98]:
#  1413 out of 1413 + 230 predictions have been predicted correctly

In [99]:
from sklearn.metrics import accuracy_score

In [100]:
accuracy = accuracy_score(Y_test, y_pred)

In [101]:
accuracy

0.9826555023923444